In [1]:
from metaflow import Flow, get_metadata
import re
import string

print("Current metadata provider: %s" % get_metadata())

Current metadata provider: local@/Users/sonoffungdark/Projects/merchantizer/merchantizer


### What happened in clean.py?
The code goes through various description cleaning actions, outlined in the previous steps.

In [2]:
run = Flow('DescriptionCleanFlow').latest_successful_run
print("Using run: %s" % str(run))

cleaned_df = run.data.cleaned_df

Using run: Run('DescriptionCleanFlow/1713994248757393')


### View Cleaned Format Across 3 Important Merchants
Next, let's take a look at the updated format after the data cleaning:

In [3]:
cleaned_df[cleaned_df["cleaned_description"].str.contains("target")]

,merchantName,description,cleaned_description
94,TARGET 00008243,TARGET 00008243,target 00008243
125,TARGET 00024299,TARGET 00024299,target 00024299
163,TARGET 00010520,TARGET 00010520,target 00010520
248,TARGET.COM *,TARGET.COM *,target *
272,TARGET.COM *,TARGET.COM *,target *
...,...,...,...
9999124,TARGET T- 4522 Fredric,TARGET T- 4522 Fredric,target t- 4522 fredric
9999615,TARGET 00010181,TARGET 00010181,target 00010181
9999709,TARGET.COM *,TARGET.COM *,target *
9999750,TARGET 00020156,TARGET 00020156,target 00020156


In [4]:
cleaned_df[cleaned_df["cleaned_description"].str.contains("doordash")]

,merchantName,description,cleaned_description
23,DD DOORDASH WINGSTOP,DD DOORDASH WINGSTOP,doordash* doordash wingstop
148,DOORDASH*APPLEBEES GRI,DOORDASH*APPLEBEES GRI,doordash*applebees gri
209,DD *DOORDASH THAICAFE,DD *DOORDASH THAICAFE,doordash* *doordash thaicafe
240,DD DOORDASH FAMILYGAR,DD DOORDASH FAMILYGAR,doordash* doordash familygar
267,DD DOORDASH LACATRINA,DD DOORDASH LACATRINA,doordash* doordash lacatrina
...,...,...,...
9999828,CASH APP*TANESE*ADD,CASH APP*TANESE*ADD,cashapp* *tanese*adoordash*
9999831,CASH APP*JASMINE*ADD C,CASH APP*JASMINE*ADD C,cashapp* *jasmine*adoordash* c
9999911,APPLE CASH BALANCE ADD,APPLE CASH BALANCE ADD,apple cash balance adoordash*
9999930,CASH APP*TY TY*ADD CAS,CASH APP*TY TY*ADD CAS,cashapp* *ty ty*adoordash* cas


In [5]:
cleaned_df[cleaned_df["cleaned_description"].str.contains("amazon")]

,merchantName,description,cleaned_description
1200,NaN,AMZN Mktp US*A25LP6O83,amazon mktp us*a25lp6o83
1213,NaN,Amazon Prime*TP26A9YK0,amazon prime*tp26a9yk0
1223,NaN,AMAZON.COM,amazon
1226,NaN,AMAZON.COM,amazon
1278,NaN,AMAZON.COM,amazon
...,...,...,...
9999654,AMZN Mktp US*TR0OT7ZK2,AMZN Mktp US*TR0OT7ZK2,amazon mktp us*tr0ot7zk2
9999666,AMZN Mktp US*FI1A763Z3,AMZN Mktp US*FI1A763Z3,amazon mktp us*fi1a763z3
9999684,AMZN Mktp US*TO0KA8KO1,AMZN Mktp US*TO0KA8KO1,amazon mktp us*to0ka8ko1
9999945,AMZN Mktp US*TP3V52Q12,AMZN Mktp US*TP3V52Q12,amazon mktp us*tp3v52q12


### Notable Issues
There are still cases where overlap between merchants exists, two prominent examples are:
1. "cashapp* *tanese*adoordash*"
2. "doordash* doordash wingstop"

This doesn't just apply to doordash and cashapp. There are certain categories that will end up colliding in this way. I have added this in as a TODO note, in clean.py.

### Building a Model to Predict Merchant

Given that our goal is to leverage machine learning to predict a merchant based upon a noisy description string, I will test the BERT langauge model's ability to do so. I have used BERT for topic modeling before, and have found it to work well for tasks like this. Here is a link to the approach/package I will lean into to achieve this:

https://maartengr.github.io/BERTopic/getting_started/zeroshot/zeroshot.html

This particular approach (zero-shot topic modeling), allows you to feed known topics in during the model fitting. This ensures important companies (however important is defined) get their own topic.